### Sunrise model
based on a labeled weather dataset, predict whether the next sunrise will be beautiful
- takes in weather vector for entire day for 365 days in COS AND label of good or bad

In [1]:
import torch
import os 
import numpy as np
import matplotlib.pyplot as plt

# dependencies: python 3.9, numpy 1.19.5

/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- load in weather data using only valid columns, leaving out the last column (wind chill label)
- create a tensor from all data

In [2]:
input_path = "prepped_data_1.csv"
weather_data = np.genfromtxt(input_path,delimiter = ",", skip_header = 1, filling_values=0.0)
weather_tensor = torch.tensor(weather_data).float()


read in as a tensor dataset

In [3]:
# get labels from the sunrise, sunset columns

sunrise_labels = weather_tensor[:, -2]
sunset_labels = weather_tensor[:, -1]

data = weather_tensor[:,:-2] # all weather data except sunrise and sunset labels

weather_dataset = torch.utils.data.TensorDataset(data, sunrise_labels)
# number of columns (minus the labels columns)
xdims = weather_tensor.shape[1] - 2
# number of rows 
ydims = weather_tensor.shape[0]


# normalization

In [4]:
# calculate mean and std and do the normalization by hand 
m = torch.mean(data, dim = 0)
s = torch.std(data, dim = 0)

norm_data = (data - m)/s

norm_data

m = m.detach().numpy().tolist()
s = s.detach().numpy().tolist()

import json

mean = json.dumps(m)
std = json.dumps(s)


with open('mean.json', 'w') as f:
    f.write(mean)
with open('std.json', 'w') as f:
    f.write(std)

DataLoader 
puts data on the right device, shuffles, can use parallel programming, define batch size etc
- sees everything in an epoch
- next epoch, sees them in a different order
- suffles with 32 irows in a batch


In [5]:
loader = torch.utils.data.DataLoader(weather_dataset, shuffle=True, batch_size=32)


In [6]:
net = torch.nn.Sequential(
    torch.nn.Linear(xdims, 32), # takes in x dims columns, then funnels to a hidden layer of 32
    torch.nn.ELU(), 
    # ELU is an activation layer, like a sigmoidal function!
    torch.nn.Linear(32, 32),
    torch.nn.ELU(),
    torch.nn.Linear(32, 16),
    torch.nn.ELU(),
    torch.nn.Linear(16, 1)
)


In [7]:
# Mean squared error loss function
criterion = torch.nn.MSELoss()


In [8]:
# optimizer called Adam, using a learning rate of 1e-4
opt = torch.optim.Adam(net.parameters(), 1e-4 )


Training loop

In [9]:
for ep in range(1000):
    # every epoch sees every row exactly one time, but inputs are shuffled every epoch
    total_loss = 0.0
    
    # batch by batch
    for batch in loader:
        
        opt.zero_grad()
        
        batchX = batch[0]
        batchY = batch[1]

        pred = net(batchX)
        loss = criterion(pred, batchY)
        
        loss.backward()
        opt.step()
        total_loss += loss
    if ep%30 == 0: 
        print("--- epoch " + str(ep))
        print(total_loss)


/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


--- epoch 0
tensor(472.6337, grad_fn=<AddBackward0>)
--- epoch 30
tensor(403.9732, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

### Test

In [ ]:
# import random 
# row = random.randint(0, 365)
# test = weather_tensor[row,:]
# label = sunrise_labels[row]
# print("Real Value:")
# print(label)
# print("Predicted Value:")
# print(net(test))

In [124]:
#pip3 install coremltools==5.0b5 protobuf==3.20.1



In [93]:
## This code will print out the real, then expected for every batch of 32 
# for batch in loader:
#     batchX = batch[0]
#     batchY = batch[1]
#     print(batchY)
#     print(net(batchX))
    

### Quantize and convert to TorchScript
The process of tracing takes an example input and traces its flow through the model. You can trace the model by creating an example image input, as shown in the above code using random data. To understand the reasons for tracing and how to trace a PyTorch model, see Model Tracing.


If your model uses a data-dependent control flow, such as a loop or conditional, the traced model won't generalize to other inputs. In such cases you can experiment with applying PyTorch's JIT script (torch.jit.script) to your model as described in Model Scripting. You can also use a combination of tracing and scripting.



In [27]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    net, qconfig_spec={torch.nn.Linear}, dtype=torch.qint8
)
# set model to evaluation mode
model_dynamic_quantized.eval()
example_tensor = weather_tensor[0,:-1]
# convert to torch script
traced_script_module = torch.jit.trace(net, example_tensor)


In [28]:
out = traced_script_module(example_tensor)

### Convert to CoreML

In [30]:
import coremltools as ct

# Using image_input in the inputs parameter:
# Convert to Core ML program using the Unified Conversion API.
model = ct.convert(
    traced_script_module,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_tensor.shape)]
 )

Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion
Running MIL Common passes:   0%|                                                                      | 0/40 [00:00<?, ? passes/s]/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:107: UserWarning: Input, 'input.1', of the source model, has been renamed to 'input_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/Users/lena/opt/anaconda3/envs/sunproject/lib/python3.9/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:135: UserWarning: Output, '26', of the source model, has been renamed to 'var_26' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL Clean up passes: 100%|█████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 1279.35 passes/s]


got this message: 

Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion


ALSO: 
As an alternative, you can convert the model to a neural network by eliminating the convert_to parameter:

In [31]:
# Save the converted model.
model.save("newmodel.mlpackage")

### success! (maybe?)
next step is to try integrating it into my app:
https://developer.apple.com/documentation/coreml/integrating_a_core_ml_model_into_your_app

### Optimize for pytorch model

In [ ]:
# from torch.utils.mobile_optimizer import optimize_for_mobile
# # optimize for mobile so that we can export and use in Swift app
# torchscript_model_optimized = optimize_for_mobile(traced_script_module)

# # save as .pt 
# path = os.path.join(os.getcwd(),"model.pt")
# torchscript_model_optimized._save_for_lite_interpreter(path)


### Test

In [128]:
row = 36
test = weather_tensor[row,:]
label = labels[row]
label

tensor([1.])

In [130]:
net(test)


tensor([0.8277], grad_fn=<AddBackward0>)

NOTES FROM CORY: HOW TO SPLIT UP training, validation, testing

def train(loader):
    one tound of training with all data in the loader
    
    
def eval(loader):
    feed evyerhting in batches to model, adds up and takes the losses of the batches
    



In [85]:
# # create a batch norm layer
# bn = torch.nn.BatchNorm1d(num_features = xdims)
# # normalize 
# data = bn(data) # how to extract the std and mean 
